# Brain Mesh

## Import of  modules

In [1]:
import os.path as path

import mne
from mne import read_source_estimate

import numpy as np

In [2]:
import mne_g3d

## Data reading

In [3]:
data_path = mne.datasets.sample.data_path()

subject = 'sample'
subjects_dir = path.join(data_path, 'subjects')
act_data = path.join(data_path, 'MEG/sample/sample_audvis-meg-eeg')

stc = read_source_estimate(act_data)
print(stc.times)
#"lat", "med", "fos", "cau", "dor",
#        "ven", "fro", "par"].

fig = mne_g3d.viz.plot_source_estimates(stc, subject=subject, surface='inflated', hemi='both',
                          colormap='hot', smoothing_steps=10, 
                          transparent=False, alpha=1.0, subjects_dir=subjects_dir,
                          clim='auto', figure=None, size=800,
                          background="black", initial_time=0.23)

[0.   0.01 0.02 0.03 0.04 0.05 0.06 0.07 0.08 0.09 0.1  0.11 0.12 0.13
 0.14 0.15 0.16 0.17 0.18 0.19 0.2  0.21 0.22 0.23 0.24]
Morphing data...
    Left-hemisphere map read.
    Right-hemisphere map read.
    Left-hemisphere map read.
    Right-hemisphere map read.
    10 smooth iterations done.
    10 smooth iterations done.
[done]
